In [ ]:
import tensorflow as tf
import scipy
import numpy as np
import time

#import models
%run -n models.ipynb

In [ ]:
flags = tf.app.flags
FLAGS = flags.FLAGS
# 0.1 for cnn2, 1.0 for fgcnn2
flags.DEFINE_float('learning_rate', 1.0, 'Initial learning rate.')
flags.DEFINE_integer('batch_size', 100, 'Batch size.')
flags.DEFINE_float('regularization', 0, 'L2 regularizations of weights and biases.')

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data_mnist", one_hot=False)

# Feature graph

In [ ]:
def grid(m):
    M = m**2
    x = np.linspace(0,1,m)
    y = np.linspace(0,1,m)
    xx, yy = np.meshgrid(x, y)
    z = np.empty((M,2))
    z[:,0] = xx.reshape(M)
    z[:,1] = yy.reshape(M)
    return z
    
def graph(z, k=4):
    M = z.shape[0]

    # Compute pairwise distances.
    d = scipy.spatial.distance.pdist(z, 'euclidean')
    d = scipy.spatial.distance.squareform(d)

    # k-NN graph.
    idx = np.argsort(d)[:,1:k+1]
    d.sort()
    d = d[:,1:k+1]

    # Weights.
    sigma2 = np.mean(d[:,-1])**2
    d = np.exp(- d**2 / sigma2)

    # Weight matrix.
    I = np.arange(0, M).repeat(k)
    J = idx.reshape(M*k)
    V = d.reshape(M*k)
    W = scipy.sparse.coo_matrix((V, (I, J)), shape=(M, M))
    
    # No self-connections.
    W.setdiag(0)

    # Non-directed graph.
    bigger = W.T > W
    W = W - W.multiply(bigger) + W.T.multiply(bigger)
    del bigger
    assert np.abs(W - W.T).mean() < 1e-10

    # CSR sparse matrix format for efficient multiplications.
    W = W.tocsr()
    W.eliminate_zeros()
    
    print("{} > {} edges".format(W.nnz, M*k))
    return W

W = graph(grid(28), k=4)

In [ ]:
def laplacian(W, normalized=True):
    """Return the Laplacian of the weigth matrix."""
    
    # Degree matrix.
    d = W.sum(axis=0)

    # Laplacian matrix.
    if not normalized:
        D = scipy.sparse.diags(d.A.squeeze(), 0)
        L = D - W
    else:
        d = 1 / np.sqrt(d)
        D = scipy.sparse.diags(d.A.squeeze(), 0)
        I = scipy.sparse.identity(d.size, dtype=D.dtype)
        L = I - D * W * D
    
    # Upper-bound on the spectrum.
    if normalized:
        lmax = 2
    else:
        lmax = scipy.sparse.linalg.eigsh(L, k=1, which='LM', return_eigenvectors=False)[0]
    
    assert np.abs(L - L.T).mean() < 1e-10
    return L, lmax

L, lmax = laplacian(W, True)

In [ ]:
def fourier(L):

    def sort(lamb, U):
        idx = lamb.argsort()
        return lamb[idx], U[:,idx]

    t_start = time.process_time()
    lamb, U = np.linalg.eig(L.toarray())
    lamb, U = sort(lamb, U)
    print('Execution time: {:.2f}s'.format(time.process_time() - t_start))
    return lamb, U

lamb, U = fourier(L)
print('Spectrum in [{:1.2e}, {:1.2e}]'.format(lamb[0], lamb[-1]))

# Neural network

In [ ]:
#x = tf.placeholder(tf.float32, (FLAGS.batch_size, 784))
#y = tf.placeholder(tf.int32, (FLAGS.batch_size))
x = tf.placeholder(tf.float32, (None, 784))
y = tf.placeholder(tf.int32, (None))

model = fc1()
model = fc2(nhiddens=100)
model = cnn2(K=5, F=10)  # K=28 is equivalent to filtering with fgcnn.
model = fgcnn2(U, F=10)

# Construct computational graph
logits = model.inference(x)
loss = model.loss(logits, y, FLAGS.regularization)
train_op = model.training(loss, FLAGS.learning_rate)
eval_correct = model.evaluation(logits, y)

# Train
t_start = time.process_time()
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
for i in range(int(1e4)):
    batch_xs, batch_ys = mnist.train.next_batch(FLAGS.batch_size)
    sess.run(train_op, feed_dict={x: batch_xs, y: batch_ys})
print('Training time: {:.2f}s'.format(time.process_time() - t_start))

# Evaluate
ncorrects = sess.run(eval_correct, feed_dict={x: mnist.test.images, y: mnist.test.labels})
precision = ncorrects / mnist.test.num_examples
print('Precision: {:.2f}% ({:d} / {:d})'.format(precision*100, ncorrects, mnist.test.num_examples))
sess.close()